# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
    classes = listdir(ROBOT_POSE_DATA_DIR)
print classes
print len(classes)

['Knee', 'Crouch', 'StandInit', 'Sit', 'Right', 'Left', 'Belly', 'Stand', 'Frog', 'HeadBack', 'Back']
11


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    print "Added %i datasets"%len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
for i in range(len(classes)):
    d, t = load_pose_data(i)
    for j in range(len(d)):
        all_data.append(d[j])
        all_target.append(t[j])

print 'total number of data and targets: ', len(all_data), len(all_target)

Added 10 datasets
Added 30 datasets
Added 52 datasets
Added 26 datasets
Added 11 datasets
Added 20 datasets
Added 19 datasets
Added 11 datasets
Added 10 datasets
Added 10 datasets
Added 23 datasets
total number of data and targets:  222 222


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
rest = int(len(all_data) * 0.3)
print str(n_training_data)+"(70%) +"+str(rest)+"(30%) ="+str(n_training_data+rest)

155(70%) +66(30%) =221


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
training_data = np.array(training_data)
all_data = np.array(all_data)
all_target = np.array(all_target)
clf.fit(all_data[training_data], all_target[training_data])
#print all_data

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [8]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []
# YOUR CODE HERE
expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])
evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         6
          1       0.97      1.00      0.98        28
          2       1.00      1.00      1.00        34
          3       1.00      1.00      1.00        17
          4       1.00      0.89      0.94         9
          5       1.00      1.00      1.00        15
          6       1.00      1.00      1.00        16
          7       1.00      1.00      1.00         3
          8       1.00      1.00      1.00         5
          9       1.00      1.00      1.00         8
         10       1.00      1.00      1.00        14

avg / total       0.99      0.99      0.99       155


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0  0]
 [ 0  0 34  0  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  0  0]
 [ 0  1  0  0  8  0  0  0  0  0  0]
 [ 0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[permutation[n_training_data:]])
expected = all_target[permutation[n_training_data:]]
evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         4
          1       1.00      1.00      1.00         2
          2       1.00      1.00      1.00        18
          3       1.00      1.00      1.00         9
          4       1.00      1.00      1.00         2
          5       1.00      1.00      1.00         5
          6       1.00      1.00      1.00         3
          7       1.00      1.00      1.00         8
          8       1.00      1.00      1.00         5
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         9

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[ 4  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [14]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [16]:

print clf2

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
